# Домашнее задание 1: ML Pipeline с DVC и MLflow

## Цель
Создание воспроизводимого ML-pipeline для классификации медицинских консультаций с использованием датасета ChatDoctor

## Структура проекта
```
HW1/
├── data/
│   ├── raw/          # Исходные данные (DVC)
│   └── processed/    # Обработанные данные (DVC)
├── src/
│   ├── prepare.py    # Подготовка данных
│   └── train.py      # Обучение модели
├── models/           # Сохраненные модели (DVC)
├── dvc.yaml          # Описание пайплайна
├── params.yaml       # Гиперпараметры
└── requirements.txt  # Зависимости
```

## Шаг 1: Установка зависимостей

Установим все необходимые библиотеки для работы с DVC, MLflow и ML

In [ ]:
%pip install -q pandas numpy scikit-learn mlflow dvc kagglehub joblib matplotlib seaborn pyyaml

## Шаг 2: Загрузка датасета ChatDoctor

Загружаем датасет с Kaggle и сохраняем в папку data/raw/

In [ ]:
import kagglehub
import os
import shutil

os.makedirs("data/raw", exist_ok=True)

print("Загрузка датасета ChatDoctor...")
path = kagglehub.dataset_download("punyaslokaprusty/chatdoctor")
print(f"Датасет загружен в: {path}")

csv_files = [f for f in os.listdir(path) if f.endswith('.csv')]
print(f"\nНайдены файлы: {csv_files}")

if csv_files:
    source_file = os.path.join(path, csv_files[0])
    target_file = "data/raw/chatdoctor.csv"
    shutil.copy(source_file, target_file)
    print(f"\nДатасет скопирован в: {target_file}")
else:
    print("CSV файлы не найдены!")

import kagglehub
import os
import shutil
import pandas as pd

os.makedirs("data/raw", exist_ok=True)

print("Загрузка датасета ChatDoctor...")
path = kagglehub.dataset_download("punyaslokaprusty/chatdoctor")
print(f"Датасет загружен в: {path}")

files = os.listdir(path)
print(f"\nФайлы в датасете: {files}")

for file in files:
    source = os.path.join(path, file)
    if os.path.isfile(source):
        target = os.path.join("data/raw", file)
        shutil.copy(source, target)
        print(f"Скопирован: {file}")
        
        if file.endswith('.csv'):
            df = pd.read_csv(target, nrows=5)
            print(f"\nСтруктура {file}:")
            print(f"Колонки: {list(df.columns)}")
            print(f"Размер: {pd.read_csv(target).shape}")
            print(f"\nПервые строки:")
            print(df.head(2))

print("\nДанные готовы к обработке!")

## Шаг 2.1: Анализ структуры данных

Изучаем структуру датасета для понимания формата и выбора стратегии обработки

In [ ]:
import pandas as pd
import json
import os

print("Анализ загруженных данных:\n")

for file in os.listdir("data/raw"):
    filepath = os.path.join("data/raw", file)
    
    if file.endswith('.csv'):
        df = pd.read_csv(filepath)
        print(f"CSV файл: {file}")
        print(f"  Размер: {df.shape}")
        print(f"  Колонки: {list(df.columns)}")
        print(f"  Пример данных:\n{df.head(2)}\n")
        
    elif file.endswith('.json'):
        with open(filepath, 'r', encoding='utf-8') as f:
            try:
                data = json.load(f)
                if isinstance(data, list):
                    print(f"JSON файл: {file}")
                    print(f"  Количество записей: {len(data)}")
                    print(f"  Пример записи: {data[0]}\n")
            except:
                print(f"JSON файл (построчный): {file}")

print("\nСтратегия обработки:")
print("1. Извлечение текста вопросов пациентов и ответов врачей")
print("2. Создание меток: нужен ли офлайн-визит (на основе ключевых слов)")
print("3. Формирование датасета: [text, label]")

In [ ]:
!git init

!dvc init

!git add .gitignore .dvc/config .dvc/.gitignore
!git commit -m "Initialize Git and DVC"

print("\nGit и DVC инициализированы!")

## Шаг 4: Версионирование данных через DVC

Добавляем датасет под контроль DVC и коммитим метаданные в Git

In [ ]:
!dvc add data/raw/chatdoctor.csv

!git add data/raw/chatdoctor.csv.dvc data/raw/.gitignore
!git commit -m "Add raw dataset via DVC"

print("\nДатасет добавлен в DVC!")
print("Создан файл data/raw/chatdoctor.csv.dvc с метаданными")

## Шаг 5: Создание DVC пайплайна

Создаем dvc.yaml с двумя стадиями: prepare и train

In [ ]:
dvc_yaml_content = """stages:
  prepare:
    cmd: python src/prepare.py
    deps:
      - src/prepare.py
      - data/raw/chatdoctor.csv
    params:
      - prepare.test_size
      - prepare.random_state
      - prepare.sample_size
    outs:
      - data/processed/train.csv
      - data/processed/test.csv

  train:
    cmd: python src/train.py
    deps:
      - src/train.py
      - data/processed/train.csv
      - data/processed/test.csv
    params:
      - train.model_type
      - train.n_estimators
      - train.max_depth
      - train.random_state
      - train.max_features
    outs:
      - models/model.pkl
"""

with open("dvc.yaml", "w") as f:
    f.write(dvc_yaml_content)

print("Файл dvc.yaml создан!")
print("\nСодержимое:")
print(dvc_yaml_content)

## Шаг 6: Запуск DVC пайплайна

Запускаем полный пайплайн: prepare → train

In [ ]:
!dvc repro

## Шаг 7: Версионирование обработанных данных и модели

Добавляем в DVC обработанные данные и модель, фиксируем изменения

In [ ]:
!git add dvc.yaml dvc.lock params.yaml requirements.txt README.md .gitignore
!git add src/prepare.py src/train.py
!git add data/processed/.gitignore models/.gitignore
!git add data/processed/train.csv.dvc data/processed/test.csv.dvc models/model.pkl.dvc

!git commit -m "Add DVC pipeline with prepare and train stages"

print("\nВсе изменения закоммичены в Git!")
print("Метаданные данных и модели под контролем DVC")

## Шаг 8: Настройка удаленного хранилища DVC

Создаем локальное хранилище для данных и моделей

In [ ]:
import os

os.makedirs("../dvc-storage", exist_ok=True)

!dvc remote add -d localstorage ../dvc-storage
!dvc push

!git add .dvc/config
!git commit -m "Configure DVC remote storage"

print("\nDVC remote storage настроен!")
print("Данные и модели загружены в ../dvc-storage")

## Шаг 9: Просмотр результатов в MLflow

Запускаем MLflow UI для просмотра экспериментов, метрик и моделей

In [ ]:
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")

print("Для просмотра MLflow UI выполните в терминале:")
print("\n  mlflow ui --backend-store-uri sqlite:///mlflow.db")
print("\nЗатем откройте в браузере: http://127.0.0.1:5000")
print("\n" + "="*60)

experiments = mlflow.search_experiments()
print("\nДоступные эксперименты:")
for exp in experiments:
    print(f"  - {exp.name} (ID: {exp.experiment_id})")

runs = mlflow.search_runs(experiment_names=["chatdoctor_classification"])
if not runs.empty:
    print(f"\nЗапущено экспериментов: {len(runs)}")
    print("\nПоследний эксперимент:")
    latest = runs.iloc[0]
    print(f"  Accuracy:  {latest['metrics.accuracy']:.4f}")
    print(f"  Precision: {latest['metrics.precision']:.4f}")
    print(f"  Recall:    {latest['metrics.recall']:.4f}")
    print(f"  F1-score:  {latest['metrics.f1_score']:.4f}")

## Шаг 10: Проверка воспроизводимости

Проверяем структуру проекта и готовность к воспроизведению

In [ ]:
import os

print("Проверка структуры проекта:")
print("="*60)

required_files = [
    "dvc.yaml",
    "params.yaml",
    "requirements.txt",
    "README.md",
    ".gitignore",
    "src/prepare.py",
    "src/train.py"
]

for file in required_files:
    exists = "✓" if os.path.exists(file) else "✗"
    print(f"{exists} {file}")

print("\n" + "="*60)
print("\nПроект готов к публикации!")
print("\nДля воспроизведения на другой машине:")
print("  1. git clone <repo>")
print("  2. cd <repo>")
print("  3. pip install -r requirements.txt")
print("  4. dvc pull")
print("  5. dvc repro")
print("\nMLflow UI:")
print("  mlflow ui --backend-store-uri sqlite:///mlflow.db")

## Итоги и оптимизация для MacBook Pro M4

### Реализованная задача
**Бинарная классификация медицинских консультаций:**
- Класс 0: Консультация решается онлайн
- Класс 1: Требуется офлайн-визит к врачу

### ML Pipeline
1. **prepare.py**: Парсинг данных ChatDoctor, автогенерация меток, train/test split
2. **train.py**: TF-IDF + RandomForest, логирование метрик в MLflow
3. **dvc.yaml**: Автоматизация через `dvc repro`

### Оптимизация для M4
- `n_jobs=-1`: использование всех ядер CPU
- TF-IDF max_features=5000: баланс качества и скорости
- Sample size 10000: быстрое обучение для экспериментов
- `max_depth=15`: предотвращение переобучения

### Альтернативные модели для M4
- **LogisticRegression**: быстрая, хорошо работает с текстом
- **SGDClassifier**: для очень больших датасетов
- **ComplementNB**: эффективен для несбалансированных классов

### Следующие шаги
1. Загрузить в GitHub
2. Настроить DVC remote (локальный или S3)
3. Экспериментировать с параметрами через `params.yaml`
4. Добавить evaluate стадию (опционально)